In [4]:
from time import time
import os
import sys
sys.path.append('../')

from utils.codification_cnn import CNNLayer, NNLayer, ChromosomeCNN, FitnessCNN, FitnessCNNParallel
from utils.datamanager import DataManager
from utils.lr_finder import LRFinder
from time import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras

In [ ]:
fitness_cnn = FitnessCNN()

# dataset params:
data_folder = '/home/daniel/datasets/MNIST_variations'
classes = []

# Fitness params
epochs = 15
batch_size = 128
verbose = 1
redu_plat = False
early_stop = 0
warm_up_epochs= 5
base_lr = 0.002
smooth = 0.1
cosine_dec = True
lr_find = True

dataset = 'MRDBI'
dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder)
data = dm.load_data()
print(data[0][0].shape)

fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                   epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                   warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find)

In [ ]:
l1_2 = CNNLayer(86, (3,5), 'leakyreLu', 0.262, 1)
l2_2 = CNNLayer(84, (5,3), 'leakyreLu', 0.319, 1)
l3_2 = CNNLayer(243, (1,3), 'prelu', 0.322, 1)
l4_2 = NNLayer(948, 'sigmoid', 0.467)
l5_2 = NNLayer(780, 'sigmoid', 0.441)
base_model = ChromosomeCNN([l1_2, l2_2, l3_2], [l4_2, l5_2], fitness_cnn)

l1_1 = CNNLayer(32, (3,3), 'relu', 0.2, 1)
l2_1 = CNNLayer(64, (3,3), 'relu', 0.2, 1)
l3_1 = NNLayer(512, 'relu', 0.5)
l4_1 = NNLayer(512, 'relu', 0.5)
simple_model = ChromosomeCNN([l1_1, l2_1], [l3_1, l4_1], fitness_cnn)

models = {'simple': simple_model, 'base':base_model}

In [ ]:
from keras import Input, Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model
from keras.layers import Conv2D, PReLU, LeakyReLU, Dropout, MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.layers.merge import concatenate

from utils.utils import smooth_labels, WarmUpCosineDecayScheduler

In [33]:
init_conv = True
k_sizes_0 = [1,3,5]
print(data[0][0][0].shape)

(28, 28, 1)


In [34]:
keras.backend.clear_session()

inp = Input(shape=data[0][0][0].shape)
x = inp
if init_conv:
    x = Conv2D(32, 1, activation='relu', padding='same')(x)
layer_0 = []
for k in k_sizes_0:
    layer_0.append(Conv2D(32, k, activation='relu', padding='same')(x))
x = concatenate(layer_0)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)
x = Dropout(0.3)(x)

layer_1 = []
for k in k_sizes_0:
    layer_1.append(Conv2D(64, k, activation='relu', padding='same')(x))
x = concatenate(layer_1)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)
x = Dropout(0.3)(x)

#x = Conv2D(256, 3, activation='relu', padding='same')(x)
#x = BatchNormalization()(x)
#x = MaxPooling2D()(x)
#x = Dropout(0.3)(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.45)(x)
x = Dense(10, activation='softmax')(x)

model = Model(inputs=inp, outputs=x)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 32)   64          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 64)   2112        conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 28, 28, 64)   18496       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

In [35]:
epochs = 75
warmup_epochs = 5
batch_size=256
cosine_decay = True

callbacks = []
# Create the Learning rate scheduler.
total_steps = int(epochs * data[0][1].shape[0] / batch_size)
warm_up_steps = int(warmup_epochs * data[0][1].shape[0] /batch_size)
base_steps = total_steps * (not cosine_decay)
warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=0.001,
                                        total_steps=total_steps,
                                        warmup_learning_rate=0.0,
                                        warmup_steps=warm_up_steps,
                                        hold_base_rate_steps=base_steps)
callbacks.append(warm_up_lr)


h = model.fit(data[0][0], data[0][1],
      batch_size=batch_size,
      epochs=epochs,
      verbose=1,
      validation_data=(data[2][0], data[2][1]),
             callbacks=callbacks)
print("Test:", model.evaluate(data[1][0], data[1][1], verbose=0)[1])

Train on 9600 samples, validate on 2400 samples
Epoch 1/75
9600/9600 [==============================] - 21s 2ms/step - loss: 3.4242 - acc: 0.1284 - val_loss: 2.2332 - val_acc: 0.1837
Epoch 2/75
9600/9600 [==============================] - 13s 1ms/step - loss: 2.2429 - acc: 0.1742 - val_loss: 2.0816 - val_acc: 0.2729
Epoch 3/75
9600/9600 [==============================] - 13s 1ms/step - loss: 2.1134 - acc: 0.2470 - val_loss: 1.8663 - val_acc: 0.3350
Epoch 4/75
9600/9600 [==============================] - 13s 1ms/step - loss: 2.0064 - acc: 0.2984 - val_loss: 1.7979 - val_acc: 0.3396
Epoch 5/75
9600/9600 [==============================] - 13s 1ms/step - loss: 1.9422 - acc: 0.3362 - val_loss: 1.6854 - val_acc: 0.3962
Epoch 6/75
9600/9600 [==============================] - 13s 1ms/step - loss: 1.8652 - acc: 0.3755 - val_loss: 1.5668 - val_acc: 0.4492
Epoch 7/75
9600/9600 [==============================] - 14s 1ms/step - loss: 1.7767 - acc: 0.4288 - val_loss: 1.5249 - val_acc: 0.4633
Epoch 8

Epoch 61/75
9600/9600 [==============================] - 15s 2ms/step - loss: 0.7068 - acc: 0.9715 - val_loss: 0.6222 - val_acc: 0.8329
Epoch 62/75
9600/9600 [==============================] - 14s 1ms/step - loss: 0.7035 - acc: 0.9746 - val_loss: 0.6200 - val_acc: 0.8342
Epoch 63/75
9600/9600 [==============================] - 13s 1ms/step - loss: 0.7070 - acc: 0.9714 - val_loss: 0.6184 - val_acc: 0.8346
Epoch 64/75
9600/9600 [==============================] - 13s 1ms/step - loss: 0.7041 - acc: 0.9739 - val_loss: 0.6209 - val_acc: 0.8350
Epoch 65/75
9600/9600 [==============================] - 13s 1ms/step - loss: 0.7012 - acc: 0.9756 - val_loss: 0.6237 - val_acc: 0.8338
Epoch 66/75
9600/9600 [==============================] - 13s 1ms/step - loss: 0.6982 - acc: 0.9766 - val_loss: 0.6182 - val_acc: 0.8350
Epoch 67/75
9600/9600 [==============================] - 13s 1ms/step - loss: 0.6988 - acc: 0.9727 - val_loss: 0.6237 - val_acc: 0.8338
Epoch 68/75
9600/9600 [=========================